In [ ]:
import os
os.chdir('C:\\Github')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter
import statistics
import random
import csv
import os
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [ ]:
def class_to_coordinate(a):
    table = np.array([\
        [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41]])
    
    x = np.argwhere(table == a)[0][1]
    y = np.argwhere(table == a)[0][0]
    
    coordinate = [x,y]
    
    return coordinate

def add_weight(particle_weight_map, pos, particle_num, alpha, c, spread = 3):
    
    # add weight to target position, and spread to neighbor
    alpha_1 = alpha*c
    alpha_2 = alpha_1*c
    alpha_3 = alpha_2*c
    
    for direction in range(4):
        particle_weight_map = add_weight_dir(particle_weight_map, pos, direction, alpha * particle_num)
    
    x = position_list[pos][0]
    y = position_list[pos][1]
    
    if spread > 0:
        
        neighbor_1 = [[x+1, y], [x,y+1], [x-1,y], [x,y-1]]
        neighbor_2 = [[x+1,y+1], [x-1,y+1], [x+1,y-1], [x-1,y-1], [x+2,y], [x,y+2], [x-2,y], [x,y-2]]
        neighbor_3 = [[x+1,y+2], [x+2,y+1], [x+2,y-1], [x+1,y-2], [x-1,y-2], [x-2,y-1], [x-2,y+1], [x-1,y+2],\
                      [x+3, y], [x,y+3], [x-3,y], [x,y-3]]
        
        neighbor_list = [neighbor_1, neighbor_2, neighbor_3]
        parameter_list = [alpha_1, alpha_2, alpha_3]
                
        for i in range(spread): 
            for neighbor_pos in neighbor_list[i]:
                if neighbor_pos in position_list:
                    np = position_list.index(neighbor_pos)
                    for direction in range(4):
                        particle_weight_map = add_weight_dir(particle_weight_map, np, direction, parameter_list[i] * particle_num)
                    
    return particle_weight_map

def add_weight_dir(particle_weight_map, pos, direction, particle_num):
    coor = position_list[pos]
    particle_weight_map[coor[0]][coor[1]][direction] += particle_num
    return particle_weight_map

def proportion_init(particle_weight_map, p_count_knn):
    
    # spread particles
    w_sum = 0
    for i in range(int(p_count_knn/4)):
        for d in range(4):
            pos = position_list[ random.randint(0,33) ]
            particle_weight_map[pos[0]][pos[1]][d] += (1/p_count_knn)
            w_sum += (1/p_count_knn)
    
    return particle_weight_map

def euclidean_distance(p1,p2):
    return np.sqrt((p1[0]-p2[0])*(p1[0]-p2[0])+(p1[1]-p2[1])*(p1[1]-p2[1]))

def avg_normalization(particle_weight_map_i):
    w_sum = 0
    for pos in position_list:
        for d in range(4):
            w_sum += particle_weight_map_i[pos[0]][pos[1]][d]
            
    for pos in position_list:
        for d in range(4):
            particle_weight_map_i[pos[0]][pos[1]][d] /= w_sum
            
    return particle_weight_map_i

def min_max_normalization(particle_weight_map_i):
    Min = 9999999
    Max = 0
    for pos in position_list:
        for d in range(4):
            if particle_weight_map_i[pos[0]][pos[1]][d] > Max:
                Max = particle_weight_map_i[pos[0]][pos[1]][d]
            if particle_weight_map[pos[0]][pos[1]][d] < Min:
                Min = particle_weight_map_i[pos[0]][pos[1]][d]
    for pos in position_list:
        for d in range(4):
            particle_weight_map_i[pos[0]][pos[1]][d] = (particle_weight_map_i[pos[0]][pos[1]][d]-Min)/Max-Min
            
    return particle_weight_map_i

def normalization(particle_weight_map_i):
    return avg_normalization(particle_weight_map_i)

def particle_move(particle_weight_map):
    new_map = np.zeros((13,13,4))
    for pos in position_list: # for every particle
        
        if [pos[0] - 1, pos[1]] in position_list:
            new_map[pos[0] - 1][pos[1]][0] += particle_weight_map[pos[0]][pos[1]][0]*1 # 0: x-1
        if [pos[0], pos[1] + 1] in position_list:
            new_map[pos[0]][pos[1] + 1][1] += particle_weight_map[pos[0]][pos[1]][1]*1 # 1: y+1
        if [pos[0] + 1, pos[1]] in position_list:
            new_map[pos[0] + 1][pos[1]][2] += particle_weight_map[pos[0]][pos[1]][2]*1 # 2: x+1
        if [pos[0], pos[1] - 1] in position_list:
            new_map[pos[0]][pos[1] - 1][3] += particle_weight_map[pos[0]][pos[1]][3]*1 # 3: y+1
            
    return new_map

def show_map(particle_weight_map_i, flag, walk, step, true_position, true_heading, max_pos, statement, note):
#     particle_weight_map_i = normalization(particle_weight_map_i)
    x = particle_weight_map.shape[0]
    y = particle_weight_map.shape[1]
    
    if flag:
        show_map = np.zeros((x,y))
        for d in range(4):
            for ix in range(x):
                for iy in range(y):
                    if [ix,iy] in position_list:
                        show_map[ix][iy] += particle_weight_map_i[ix][iy][d]
                    else:
                        show_map[ix][iy] = -0.05
#         print(f'{show_map}')
        figure = plt.figure()
        axes = figure.add_subplot(111)
        caxes = axes.matshow(show_map, cmap = plt.get_cmap('Blues'))
        figure.colorbar(caxes) 
        
        plt.title(f'Step {step}, True:{true_position[i]}, Predicted:{max_pos}')

        plt.xticks(np.arange(0, 13, step=1))#设置刻度
        plt.yticks(np.arange(0, 13, step=1))
        
        axes.xaxis.tick_bottom()
        
        if not os.path.exists(f'{walk}_map_res'):
            os.makedirs(f'{walk}_map_res')
            
        if not os.path.exists(f'{walk}_map_res/{statement}') and statement != '':
            os.makedirs(f'{walk}_map_res/{statement}')
            
        plt.savefig(f'{walk}_map_res/{statement}/{step}_{note}.png', dpi=200)
        
        plt.show()
#         print(f'sum: {np.sum(show_map)}') 
    else:
        show_map = np.zeros((x,y))
        for ix in range(x):
            for iy in range(y):
                if [ix,iy] in position_list:
                    show_map[ix][iy] = particle_weight_map_i[ix][iy][true_heading]
                else:
                    show_map[ix][iy] = -1* np.mean(particle_weight_map_i)
#         print(f'{note}')
#         print(f'{show_map}')
        figure = plt.figure()
        axes = figure.add_subplot(111)
        caxes = axes.matshow(show_map, cmap = plt.get_cmap('Blues'))
        figure.colorbar(caxes) 
        
        plt.title(f'Step:{step},True:{true_position[i]},Predicted:{max_pos}')

        plt.xticks(np.arange(0, 13, step=1))#
        plt.yticks(np.arange(0, 13, step=1))
        
        axes.xaxis.tick_bottom()
        
        if not os.path.exists(f'{walk}_map_res'):
            os.makedirs(f'{walk}_map_res')
            
        if not os.path.exists(f'{walk}_map_res/{statement}') and statement != '':
            os.makedirs(f'{walk}_map_res/{statement}')
            
        plt.savefig(f'{walk}_map_res/{statement}/{step}_{note}.png', dpi=200)
        
        plt.show()

In [ ]:
best_distance_error_list = []

position_list = [[1,1],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[1,9],[1,10],
                 [6,1],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[6,9],[6,10],
                 [11,1],[11,2],[11,3],[11,4],[11,5],[11,6],[11,7],[11,8],[11,9],[11,10],
                 [11,11],[10,11],[9,11],[8,11],[7,11],[6,11],[5,11],[4,11],[3,11],[2,11],[1,11]]
# without [7,8],[7,9],[7,10],

particle_weight_map = np.zeros((13,13,4))

In [ ]:
alpha = 1
c = 0.5
statement = ''

scripted_walk = [
    'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
    'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
]

stationary = [
    'stationary_1'
]

freewalk = [
    'freewalk_1','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_8','freewalk_9'
]


walk_list = freewalk

obj_missing_rate = 1 # 1 8 6
beacon_count = 7  # 7 6 5 4 
beacon_rate = 1
# 1 GOOD 
# 2 1 Bad
# 3 2 Bad


methods_list =  [f'dev_1_A51',f'dev_1_U11',f'dev_1_U19e',f'dev_1_025',\
                 f'dev_2',f'dev_3',f'dev_4'
]

methods = 'dev_4'
data_type = methods

p_count_knn = 8000
particle_prev_count = 8000

In [ ]:
error_distribution = {}

for data_count in range(0,1):
    error_distribution[data_type] = []
    
    missing_statement = ''
    if obj_missing_rate != 1:
        missing_statement = f'_{int(obj_missing_rate*10)}'

    Converge = {}
    for W in walk_list:
        Converge[W] = []

    spread = 3

    for walk in walk_list:

        file_name = f'./image_to_pf/92589_itp_{data_type}_{walk}_omr_{obj_missing_rate}_bc_{beacon_count}_br_{beacon_rate}.csv'
        
        data = pd.read_csv(file_name, delimiter=',')

        knn_candidates = []
        true_position = []
        
        # Read knn result and ground truth from the file
        for index, row in data.iterrows():
            ans_list = eval(row[data_type])
            label = class_to_coordinate(row['label'])
            knn_candidates.append(Counter(ans_list))
            true_position.append(label)

        particle_weight_map = np.zeros((13,13,4))

        predict_pos_list = []

        distance_error_list = []

        predicted_heading_list = [-1] # 第一個方向不知道

        particle_weight_map = proportion_init(particle_weight_map, p_count_knn)

        heading_list = []

        for i in range(len(knn_candidates)): # for 每個 KNN 預測的位置

            max_weight = 0
            max_pos = [0, 0]

            if i == 0: # if 第一個位置 (是特例，沒有前一個位置)
                # 將該位置候選人以票數做 weight, 更新 partcle weight

                candidate_list = list(knn_candidates[i].keys()) # 每一輪預測KNN的參考位置 ( all 候選人 )

                candidate_vote_list = list(knn_candidates[i].values()) # 每一輪預測 KNN 的參考位置的次數 ( all 候選人票數 )

                all_vote = sum(candidate_vote_list) # 總票數

                # 由得票數設定 weight
                for k in range(len(candidate_vote_list)): # 每一個參考位置的比例分配
                    candidate_vote_list[k] /= all_vote

                # 把 weight佈到 map上
                for j, element in enumerate(candidate_list): #每一輪KNN的每一個參考位置
                    if class_to_coordinate(element) in position_list:
                        position_list_index = position_list.index(class_to_coordinate(element))
                        particle_weight_map = add_weight(particle_weight_map, position_list_index, 1*candidate_vote_list[j], alpha, c, spread)

                # 找地圖重心
                weight_sum = 0
                for pos in position_list:
                    for d in range(4):
                        if max_weight < particle_weight_map[pos[0]][pos[1]][d]:
                            max_weight = particle_weight_map[pos[0]][pos[1]][d]
                            max_pos = pos

                predict_pos_list.append(max_pos)

                particle_weight_map = normalization(particle_weight_map)

                C = []
                for pos in position_list:
                    for d in range(4):
                        C.append(particle_weight_map[pos[0]][pos[1]][d]*euclidean_distance(max_pos,pos)*0.6)
                Converge[walk].append(sum(C))

                error = round(euclidean_distance(true_position[i], predict_pos_list[-1])*0.6, 3)

                KNN_res = [(class_to_coordinate(x[0]),x[1]) for x in knn_candidates[i].most_common()]
#                 fp.write(f'step: {i}\nKNN res: {KNN_res}\nTrue: {true_position[i]}, Predicted: {max_pos}, Error: {error}\n')

                distance_error_list.append(error)

            else:

                for pos in position_list:
                    for d in range(4):
                        particle_weight_map[pos[0]][pos[1]][d] *= particle_prev_count

                candidate_list = list(knn_candidates[i].keys()) # 每一輪預測KNN的參考位置 ( all 候選人 )

                candidate_vote_list = list(knn_candidates[i].values()) # 每一輪預測 KNN 的參考位置的次數 ( all 候選人票數 )

                all_vote = sum(candidate_vote_list) # 總票數

                # 由得票數設定 weight
                for k in range(len(candidate_vote_list)): # 每一個參考位置的比例分配
                    candidate_vote_list[k] /= all_vote

                for j,element in enumerate(candidate_list): #每一輪 KNN 的每一個參考位置
                    if class_to_coordinate(element) in position_list:
                        position_list_index = position_list.index(class_to_coordinate(element))
                        particle_weight_map = add_weight(particle_weight_map, position_list_index, \
                                                         p_count_knn*candidate_vote_list[j], alpha, c, spread)

                # 找地圖重心
                weight_sum = 0
                for pos in position_list:
                    for d in range(4):
                        if max_weight < particle_weight_map[pos[0]][pos[1]][d]:
                            max_weight = particle_weight_map[pos[0]][pos[1]][d]
                            max_pos = pos

                predict_pos_list.append(max_pos)

                particle_weight_map = normalization(particle_weight_map)

                C = []
                for pos in position_list:
                    for d in range(4):
                        C.append(particle_weight_map[pos[0]][pos[1]][d]*euclidean_distance(max_pos,pos)*0.6)
                Converge[walk].append(sum(C))


                error = round(euclidean_distance(true_position[i], predict_pos_list[-1])*0.6, 3)

                KNN_res = [(class_to_coordinate(x[0]),x[1]) for x in knn_candidates[i].most_common()]
#                 fp.write(f'step: {i}\nKNN res: {KNN_res}\nTrue: {true_position[i]}, Predicted: {max_pos}, Error: {error}\n')

                distance_error_list.append(error)
                    
            error_distribution[data_type] += distance_error_list

In [ ]:
file_name_mde = f'./Result/MDE_92589_itp_{methods}_{walk_list[0]}_omr_{obj_missing_rate}_bc_{beacon_count}_br_{beacon_rate}.csv'
with open(file_name_mde, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['method','MDE'])

    for data_count in range(0,1):
        mde = sum(error_distribution[data_type])/len(error_distribution[data_type])
        mde = round(mde, 4)
        print(f'{data_type},{mde}') # w/i
        #print(error_distribution[data_type])
        writer.writerow([data_type,mde])

In [ ]:
file_name_cdf = f'./Result/CDF_92589_itp_{methods}_{walk_list[0]}_omr_{obj_missing_rate}_bc_{beacon_count}_br_{beacon_rate}.csv'
with open(file_name_cdf, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    x = np.arange(0,8.2,0.2)
    
    writer.writerow(['CDF']+x.tolist())

    for data_count in range(0,1):
        data = pd.DataFrame(error_distribution[data_type])
        prob = []
        
        for i in x:
            result = data.where(data<=i).count().sum()
            prob.append(result/len(data))

        print(f'{data_type}',*prob, sep=", ")
        writer.writerow([data_type]+prob)

In [ ]:
file_name_var = f'./Result/VAR_92589_itp_{methods}_{walk_list[0]}_omr_{obj_missing_rate}_bc_{beacon_count}_br_{beacon_rate}.csv'
with open(file_name_var, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['method','Var'])
    
    for data_count in range(0,1):
        print(f'{data_type},{np.var(error_distribution[data_type])}')
        writer.writerow([data_type,np.var(error_distribution[data_type])])